# Examples to use Groupby, pivot.table in Pandas

In [1]:
# Import packages
from pathlib import Path
import numpy as np
import pandas as pd

# Load the data
Dataset sample of the Melbourn housing dataset available on Kaggle

In [14]:
dir_name = Path.cwd()
file = 'Data/melb_data.csv' # file name
# Read the csv.file
df = pd.read_csv(dir_name/file, usecols=['Price', 'Landsize', 'Distance', 'Type', 'Regionname']) # Select columns
# Filter data - Price < 3M AND Landsize < 1200
df = df[(df.Price<3_000_000) & (df.Landsize<1200)].reset_index(drop=True)
df.head()

,Type,Price,Distance,Landsize,Regionname
0,h,1480000.0,2.5,202.0,Northern Metropolitan
1,h,1035000.0,2.5,156.0,Northern Metropolitan
2,h,1465000.0,2.5,134.0,Northern Metropolitan
3,h,850000.0,2.5,94.0,Northern Metropolitan
4,h,1600000.0,2.5,120.0,Northern Metropolitan


## Basic groupby function
Perform the average distance for each category in df.Type (h, t, u)

In [15]:
df[['Type', 'Distance']].groupby('Type').mean() # First select cols = Type, Distance, second perform groupby 'Type'

,Distance
Type,
h,10.972498
t,9.892477
u,7.832750


Type h is further away from the central district (reference point fro